# 

In [1]:
import pyreadstat
_, meta = pyreadstat.read_dta("IAIR7EFL.DTA", metadataonly=True)
print(len(meta.column_names), "columns found")
print(meta.column_names[:25])  #just 25


5972 columns found
['caseid', 'v000', 'v001', 'v002', 'v003', 'v004', 'v005', 'v006', 'v007', 'v008', 'v008a', 'v009', 'v010', 'v011', 'v012', 'v013', 'v014', 'v015', 'v016', 'v017', 'v018', 'v019', 'v019a', 'v020', 'v021']


In [2]:
import pandas as pd

# Load file ( will upload in git ) 
df = pd.read_parquet("nfhs5_ir_thin.parquet")


n_total = len(df)
missing_before = df["bmi"].isna().sum()

# Mark impossible BMI values as missing
bad = (df["bmi"] <= 10) | (df["bmi"] >= 60)
df.loc[bad, "bmi"] = pd.NA

print("Flagged as implausible BMI:", bad.sum(), "rows")

# Save
df.to_parquet("nfhs5_ir_thin_clean.parquet", index=False)

print(df["bmi"].describe())
print("Missing BMI before → after:", missing_before, "→", df["bmi"].isna().sum(), "of", n_total)


Flagged as implausible BMI: 904 rows
count    699362.000000
mean         22.284112
std           4.308438
min          12.000000
25%          19.290000
50%          21.680000
75%          24.510000
max          59.990000
Name: bmi, dtype: float64
Missing BMI before → after: 23849 → 24753 of 724115


In [3]:
import os, pyreadstat


path = r"D:\NFHS\DataBase-Files\2019-2021\Household Member Recode-IAPR7EDT\IAPR7EFL.DTA"

print("Jupyter cwd:", os.getcwd())
print("File exists at path? ->", os.path.exists(path))

if os.path.exists(path):
    print("Size (MB):", round(os.path.getsize(path)/1e6, 1))
    _, meta_pr = pyreadstat.read_dta(path, metadataonly=True)
    print("Columns found:", len(meta_pr.column_names))
else:
    print("Path is wrong for this notebook. Paste the exact full path.")


Jupyter cwd: C:\Users\Lion\NFHS\2019-2021\Individual Recode-IAIR7EDT
File exists at path? -> True
Size (MB): 2062.0
Columns found: 530


In [4]:
import pyreadstat


path = r"D:\NFHS\DataBase-Files\2019-2021\Household Member Recode-IAPR7EDT\IAPR7EFL.DTA"


_, meta_pr = pyreadstat.read_dta(path, metadataonly=True)
names  = meta_pr.column_names
labels = meta_pr.column_labels  

def search(keys, limit=30):
    keys = [k.lower() for k in keys]
    out = []
    for n, lab in zip(names, labels):
        text = (f"{n} || {lab}").lower()
        if any(k in text for k in keys):
            out.append((n, lab))
            if len(out) >= limit:
                break
    return out

groups = {
    "JOIN + DESIGN": ["hv001","hv002","hvidx","hv005","weight","hv021","hv022","hv023","psu","strata"],
    "HEMOGLOBIN":    ["hemoglobin","haemoglobin","hb","anemia"],
    "GLUCOSE":       ["glucose","blood sugar","random","rpg"],
    "WAIST":         ["waist","abdominal","circumference"],
    "BP SYSTOLIC":   ["systolic","sbp","blood pressure"],
    "BP DIASTOLIC":  ["diastolic","dbp","blood pressure"],
}

for title, keys in groups.items():
    print(f"\n=== {title} ===")
    for n, lab in search(keys):
        print(f"{n:>14} | {lab}")



=== JOIN + DESIGN ===
         hvidx | line number
         hv001 | cluster number
         hv002 | household number
         hv005 | household sample weight (6 decimals)
         hv021 | primary sampling unit
         hv022 | sample strata for sampling errors
         hv023 | stratification used in sample design
         hv028 | household weight for male subsample (6 decimals)
         hv035 | number of eligible children for height and weight
      shweight | household weight (6 decimals) (state level)
     shmweight | household weight - male subsample (6 decimals) (state level)
         hv120 | children eligibility for height/weight and hemoglobin
           ha2 | woman's weight in kilograms (1 decimal)
          ha11 | weight/height standard deviation (dhs)
          ha12 | weight/height percent ref. median (dhs)
         ha12a | weight/height percent ref. median (fog)
         ha12b | weight/height percent ref. median (who)
          ha13 | result of measurement - height/weight
  

In [5]:
import re, pyreadstat

path = r"D:\NFHS\DataBase-Files\2019-2021\Household Member Recode-IAPR7EDT\IAPR7EFL.DTA"

_, meta_pr = pyreadstat.read_dta(path, metadataonly=True)
names  = meta_pr.column_names
labels = meta_pr.column_labels

def grep(patterns, limit=50):
    out = []
    for n, lab in zip(names, labels):
        text = f"{n} || {lab}"
        low  = text.lower()
        if any(re.search(p, low) for p in patterns):
            out.append((n, lab))
            if len(out) >= limit:
                break
    return out

patterns = [
    r"hemoglobin", r"haemoglobin", r"\bhb\b", r"\bhgb\b",
    r"anemi", r"g/dl", r"gram", r"blood haem?oglobin"
]

hits = grep(patterns, limit=100)
print("=== CANDIDATES FOR HEMOGLOBIN ===")
for n, lab in hits:
    print(f"{n:>12} | {lab}")


=== CANDIDATES FOR HEMOGLOBIN ===
       hv042 | household selected for hemoglobin
      hv253a | na - dwelling sprayed by: government worker/program
       sh72e | health insurance or scheme: community health insurance programme
       hv120 | children eligibility for height/weight and hemoglobin
         ha2 | woman's weight in kilograms (1 decimal)
        ha52 | read consent statement - hemoglobin
        ha53 | hemoglobin level (g/dl - 1 decimal)
        ha55 | result of measurement - hemoglobin
        ha56 | hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal)
        ha57 | anemia level
        ha58 | na - agrees to referral - anemia
         hc2 | child's weight in kilograms (1 decimal)
        hc52 | read consent statement - hemoglobin
        hc53 | hemoglobin level (g/dl - 1 decimal)
        hc55 | result of measurement - hemoglobin
        hc56 | hemoglobin level adjusted for altitude (g/dl - 1 decimal)
        hc57 | anemia level
        hc58 | na - agree

In [6]:
import pandas as pd, os
if os.path.exists("nfhs5_pr_thin.parquet"):
    pr_df = pd.read_parquet("nfhs5_pr_thin.parquet")
    print("Loaded pr_df:", pr_df.shape)
else:
    print("nfhs5_pr_thin.parquet not found — you can rebuild it if needed.")


Loaded pr_df: (2843917, 20)


In [7]:
import pandas as pd


pr_df["ha57"] = pd.to_numeric(pr_df.get("ha57"), errors="coerce").astype("Int16")


pr_df.to_parquet("nfhs5_pr_thin.parquet", index=False)
print("Saved nfhs5_pr_thin.parquet")
print("Shape:", pr_df.shape)


Saved nfhs5_pr_thin.parquet
Shape: (2843917, 20)


In [8]:
import pandas as pd


ir = pd.read_parquet("nfhs5_ir_thin_clean.parquet")
pr = pd.read_parquet("nfhs5_pr_thin.parquet")


for c in ["v001","v002","v003"]:
    ir[c] = pd.to_numeric(ir[c], errors="coerce").astype("Int32")
for c in ["hv001","hv002","hvidx"]:
    pr[c] = pd.to_numeric(pr[c], errors="coerce").astype("Int32")


m = ir.merge(
    pr,
    left_on=["v001","v002","v003"],
    right_on=["hv001","hv002","hvidx"],
    how="left",
    validate="many_to_one"  
)

s
biom_cols = ["shb18s","shb25s","shb29s","shb18d","shb25d","shb29d","shb74","sh305","ha53","ha56","ha57"]
coverage = m[biom_cols].notna().sum().sort_values(ascending=False)

print("Merged shape:", m.shape)
print("\nNon-missing counts (biomarkers):")
print(coverage)


m.to_parquet("nfhs5_women_biomarkers.parquet", index=False)
print("\nSaved nfhs5_women_biomarkers.parquet")


Merged shape: (724115, 37)

Non-missing counts (biomarkers):
ha56      724115
sh305     707438
ha53      702539
shb74     702492
shb18s    699184
shb18d    698971
ha57      690153
shb25s    677866
shb25d    677820
shb29s    651850
shb29d    651810
dtype: int64

Saved nfhs5_women_biomarkers.parquet


In [9]:
import pandas as pd


m = pd.read_parquet("nfhs5_women_biomarkers.parquet")

# average of 2nd & 3rd readings (uses the one that exists if the other is NaN)
sbp23 = m[["shb25s","shb29s"]].mean(axis=1, skipna=True)
dbp23 = m[["shb25d","shb29d"]].mean(axis=1, skipna=True)

# if both are NaN, fall back to 1st reading
m["sbp_clean"] = sbp23.where(~sbp23.isna(), m["shb18s"])
m["dbp_clean"] = dbp23.where(~dbp23.isna(), m["shb18d"])

# mark implausible values as missing (conservative bounds)
m.loc[~m["sbp_clean"].between(70, 260), "sbp_clean"] = pd.NA
m.loc[~m["dbp_clean"].between(40, 150), "dbp_clean"] = pd.NA

print("SBP clean non-missing:", m["sbp_clean"].notna().sum())
print("DBP clean non-missing:", m["dbp_clean"].notna().sum())

m.to_parquet("nfhs5_women_biomarkers.parquet", index=False) 
print("Saved with clean BP → nfhs5_women_biomarkers.parquet")


SBP clean non-missing: 698727
DBP clean non-missing: 698564
Saved with clean BP → nfhs5_women_biomarkers.parquet


In [10]:
import pandas as pd
m = pd.read_parquet("nfhs5_women_biomarkers.parquet")

# Hemoglobin (prefer adjusted ha56 fallback ha53)
m["hb_gdl"] = m["ha56"]
m.loc[m["hb_gdl"].isna(), "hb_gdl"] = m["ha53"]
# plausible range 5–20 g/dL
m.loc[~m["hb_gdl"].between(5, 20), "hb_gdl"] = pd.NA

# Waist circumference (cm), plausible 40–150 cm
m["waist_cm"] = m["sh305"]
m.loc[~m["waist_cm"].between(40, 150), "waist_cm"] = pd.NA

# Random glucose (mg/dL), plausible 40–500 mg/dL
m["glucose_mgdl"] = m["shb74"]
m.loc[~m["glucose_mgdl"].between(40, 500), "glucose_mgdl"] = pd.NA

# Quick coverage after cleaning
print("Hb non-missing:", m["hb_gdl"].notna().sum())
print("Waist non-missing:", m["waist_cm"].notna().sum())
print("Glucose non-missing:", m["glucose_mgdl"].notna().sum())

# Save back
m.to_parquet("nfhs5_women_biomarkers.parquet", index=False)
print("Saved with clean Hb/Waist/Glucose → nfhs5_women_biomarkers.parquet")


Hb non-missing: 52
Waist non-missing: 697562
Glucose non-missing: 690636
Saved with clean Hb/Waist/Glucose → nfhs5_women_biomarkers.parquet


In [11]:
import pandas as pd

m = pd.read_parquet("nfhs5_women_biomarkers.parquet")

print("ha56 describe (adjusted Hb):")
print(m["ha56"].describe())

print("\nha53 describe (raw Hb):")
print(m["ha53"].describe())


gt50_56 = (m["ha56"] > 50).sum()
gt50_53 = (m["ha53"] > 50).sum()
print(f"\nCounts > 50  → ha56: {gt50_56:,} | ha53: {gt50_53:,}")

# few non-missing rows to see actual numbers
print("\nSample (first 10 non-missing rows):")
print(m.loc[m["ha56"].notna() | m["ha53"].notna(), ["ha56","ha53"]].head(10))


ha56 describe (adjusted Hb):
count    724115.000000
mean        156.650629
std         187.096541
min          20.000000
25%         107.000000
50%         118.000000
75%         128.000000
max         997.000000
Name: ha56, dtype: float64

ha53 describe (raw Hb):
count    702539.000000
mean        131.367988
std         116.805252
min          20.000000
25%         107.000000
50%         117.000000
75%         127.000000
max         998.000000
Name: ha53, dtype: float64

Counts > 50  → ha56: 722,698 | ha53: 701,379

Sample (first 10 non-missing rows):
    ha56   ha53
0  106.0  117.0
1   82.0   93.0
2   83.0   94.0
3   72.0   83.0
4   93.0  104.0
5   62.0   73.0
6   63.0   74.0
7   76.0   87.0
8   85.0   96.0
9  134.0  145.0


In [12]:
import pandas as pd

m = pd.read_parquet("nfhs5_women_biomarkers.parquet")

# Scale to g/dL: 118 -> 11.8
hb56 = m["ha56"] / 10.0
hb53 = m["ha53"] / 10.0

# Prefer adjusted (ha56); fallback to raw (ha53)
m["hb_gdl"] = hb56.fillna(hb53)

# Keep plausible values only (5–20 g/dL)
m.loc[~m["hb_gdl"].between(5, 20), "hb_gdl"] = pd.NA

print("Hb non-missing after scaling & cleaning:", m["hb_gdl"].notna().sum())

m.to_parquet("nfhs5_women_biomarkers.parquet", index=False)
print("Saved with corrected Hb → nfhs5_women_biomarkers.parquet")


Hb non-missing after scaling & cleaning: 688803
Saved with corrected Hb → nfhs5_women_biomarkers.parquet


In [13]:
import pandas as pd


m = pd.read_parquet("nfhs5_women_biomarkers.parquet")

# ---- thresholds (easy to change later) ----
cfg = {
    "bmi_abn_ge": 23.0,    # abnormal if BMI >= 23 (Asian risk cut according to WHO )
    "bp_sbp_ge": 140,      # abnormal if SBP >= 140
    "bp_dbp_ge": 90,       # OR DBP >= 90
    "glucose_abn_ge": 200, # random glucose >= 200 mg/dL
    "waist_f_ge_cm": 80,   # female waist >= 80 cm (South Asian according to WHO )
    "hb_f_lt_gdl": 12.0    # hemoglobin < 12.0 g/dL
}

# ---- flags (True = abnormal; NA if metric missing) ----
m["BMI_abn"]   = (m["bmi"]             >= cfg["bmi_abn_ge"])
m["BP_abn"]    = (m["sbp_clean"]       >= cfg["bp_sbp_ge"]) | (m["dbp_clean"] >= cfg["bp_dbp_ge"])
m["GLU_abn"]   = (m["glucose_mgdl"]    >= cfg["glucose_abn_ge"])
m["WAIST_abn"] = (m["waist_cm"]        >= cfg["waist_f_ge_cm"])
m["HGB_abn"]   = (m["hb_gdl"]          <  cfg["hb_f_lt_gdl"])


for c in ["BMI_abn","BP_abn","GLU_abn","WAIST_abn","HGB_abn"]:
    m[c] = m[c].astype("boolean")


m["num_abnormal"] = m[["BMI_abn","BP_abn","GLU_abn","WAIST_abn","HGB_abn"]].sum(axis=1, min_count=1)

# "Fully normal (strict) = all five metrics present AND all five normal
present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)
none_abnormal = ~m[["BMI_abn","BP_abn","GLU_abn","WAIST_abn","HGB_abn"]].any(axis=1)
m["fully_normal_strict"] = (present_all5 & none_abnormal).astype("boolean")

print("Rows with all five measures present:", int(present_all5.sum()))
print("Saved → nfhs5_women_flags.parquet")

# Save a new file with flags
m.to_parquet("nfhs5_women_flags.parquet", index=False)


Rows with all five measures present: 682775
Saved → nfhs5_women_flags.parquet


In [14]:
import pandas as pd

m = pd.read_parquet("nfhs5_women_flags.parquet")
N = len(m)

# Recompute
present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)
n_all5 = int(present_all5.sum())

def summarize(flag_col):
    denom = int(m[flag_col].notna().sum())             # women with this metric present
    abn   = int(m[flag_col].sum(skipna=True))          # count abnormal
    pct   = (abn / denom * 100) if denom else 0.0
    return denom, abn, pct

rows = []
for name, col in [
    ("BMI abnormal",   "BMI_abn"),
    ("BP abnormal",    "BP_abn"),
    ("Glucose abnormal","GLU_abn"),
    ("Waist abnormal", "WAIST_abn"),
    ("Hemoglobin abnormal","HGB_abn"),
]:
    d, a, p = summarize(col)
    rows.append([name, d, a, round(p, 2)])

summary = pd.DataFrame(rows, columns=["Metric", "Denominator (non-missing)", "Abnormal (count)", "Abnormal (%)"])

fully_normal_count = int(m["fully_normal_strict"].sum())
fully_normal_pct_of_all5 = round(fully_normal_count / n_all5 * 100, 2) if n_all5 else 0.0
fully_normal_pct_of_all  = round(fully_normal_count / N * 100, 2)

print("Total women (IR):", N)
print("All five measures present:", n_all5)
print("\nPer-metric abnormality (unweighted):")
print(summary.to_string(index=False))

print("\nFully normal (strict):")
print("  Count:", fully_normal_count)
print("  % among those with all five present:", fully_normal_pct_of_all5)
print("  % among all women:", fully_normal_pct_of_all)


Total women (IR): 724115
All five measures present: 682775

Per-metric abnormality (unweighted):
             Metric  Denominator (non-missing)  Abnormal (count)  Abnormal (%)
       BMI abnormal                     724115            256165         35.38
        BP abnormal                     724115             71505          9.87
   Glucose abnormal                     724115              7924          1.09
     Waist abnormal                     724115            271354         37.47
Hemoglobin abnormal                     724115            393111         54.29

Fully normal (strict):
  Count: 129928
  % among those with all five present: 19.03
  % among all women: 17.94


In [15]:
import pandas as pd

m = pd.read_parquet("nfhs5_women_flags.parquet")
N = len(m)


den_bmi   = m["bmi"].notna()
den_bp    = m["sbp_clean"].notna() | m["dbp_clean"].notna()  
den_glu   = m["glucose_mgdl"].notna()
den_waist = m["waist_cm"].notna()
den_hb    = m["hb_gdl"].notna()

def pct(abn_mask, denom_mask):
    d = int(denom_mask.sum())
    a = int((abn_mask & denom_mask).sum())
    p = round(a / d * 100, 2) if d else 0.0
    return d, a, p

rows = []
rows.append(["BMI abnormal",       *pct(m["BMI_abn"].fillna(False),   den_bmi)])
rows.append(["BP abnormal",        *pct(m["BP_abn"].fillna(False),    den_bp)])
rows.append(["Glucose abnormal",   *pct(m["GLU_abn"].fillna(False),   den_glu)])
rows.append(["Waist abnormal",     *pct(m["WAIST_abn"].fillna(False), den_waist)])
rows.append(["Hemoglobin abnormal",*pct(m["HGB_abn"].fillna(False),   den_hb)])

summary = pd.DataFrame(rows, columns=["Metric","Denominator (has data)","Abnormal (count)","Abnormal (%)"])

present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)
fully_normal_count = int((m["fully_normal_strict"] == True).sum())
pct_all5 = round(fully_normal_count / int(present_all5.sum()) * 100, 2)

print("Total women:", N)
print("All five measures present:", int(present_all5.sum()))
print("\nPer-metric abnormality (correct denominators):")
print(summary.to_string(index=False))

print("\nFully normal (strict):")
print("  Count:", fully_normal_count)
print("  % among those with all five present:", pct_all5)


Total women: 724115
All five measures present: 682775

Per-metric abnormality (correct denominators):
             Metric  Denominator (has data)  Abnormal (count)  Abnormal (%)
       BMI abnormal                  699362            256165         36.63
        BP abnormal                  698956             71505         10.23
   Glucose abnormal                  690636              7924          1.15
     Waist abnormal                  697562            271354         38.90
Hemoglobin abnormal                  688803            393111         57.07

Fully normal (strict):
  Count: 129928
  % among those with all five present: 19.03


In [16]:
import pandas as pd
m = pd.read_parquet("nfhs5_women_flags.parquet")


w = pd.to_numeric(m["v005"], errors="coerce") / 1_000_000


den_bmi   = m["bmi"].notna()
den_bp    = m["sbp_clean"].notna() | m["dbp_clean"].notna()
den_glu   = m["glucose_mgdl"].notna()
den_waist = m["waist_cm"].notna()
den_hb    = m["hb_gdl"].notna()

def wshare(flag_col, denom_mask):
    
    abn = m[flag_col].fillna(False).astype(int)
    num = (abn * w).where(denom_mask, 0).sum()
    den = w.where(denom_mask, 0).sum()
    pct = float(num / den * 100) if den > 0 else 0.0
    return den, num, round(pct, 2)

rows = []
rows.append(["BMI abnormal",        *wshare("BMI_abn",   den_bmi)])
rows.append(["BP abnormal",         *wshare("BP_abn",    den_bp)])
rows.append(["Glucose abnormal",    *wshare("GLU_abn",   den_glu)])
rows.append(["Waist abnormal",      *wshare("WAIST_abn", den_waist)])
rows.append(["Hemoglobin abnormal", *wshare("HGB_abn",   den_hb)])

summary_w = pd.DataFrame(rows, columns=["Metric","Weighted denom","Weighted abn","Abnormal % (weighted)"])

# Fully normal (strict): weighted among those with all five present
present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)
fully_norm = (m["fully_normal_strict"] == True).astype(int)

den_all5_w = w.where(present_all5, 0).sum()
num_all5_w = (fully_norm * w).where(present_all5, 0).sum()
pct_all5_w = round(float(num_all5_w / den_all5_w * 100), 2) if den_all5_w > 0 else 0.0

 #weighted among all women (even if some metrics missing)
den_all_w = w.sum()
num_all_w = (fully_norm * w).sum()
pct_all_w = round(float(num_all_w / den_all_w * 100), 2)

print("Per-metric abnormality (WEIGHTED):")
print(summary_w.to_string(index=False))

print("\nFully normal (strict) — WEIGHTED:")
print("  % among those with all five present:", pct_all5_w)
print("  % among all women:", pct_all_w)


Per-metric abnormality (WEIGHTED):
             Metric  Weighted denom  Weighted abn  Abnormal % (weighted)
       BMI abnormal   692238.047527 262689.243412                  37.95
        BP abnormal   691659.557342  68332.336879                   9.88
   Glucose abnormal   682514.040153   9690.759483                   1.42
     Waist abnormal   690490.016182 283364.520297                  41.04
Hemoglobin abnormal   680847.669495 394172.621241                  57.89

Fully normal (strict) — WEIGHTED:
  % among those with all five present: 18.25
  % among all women: 17.01


In [17]:
import pandas as pd

m = pd.read_parquet("nfhs5_women_flags.parquet")
w = pd.to_numeric(m["v005"], errors="coerce") / 1_000_000


den_hb    = m["hb_gdl"].notna()
present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)

# --- BASE (Hb < 12.0) ---
HGB_abn_base = m["HGB_abn"].fillna(False).astype(int)
den_hb_w  = w.where(den_hb, 0).sum()
num_hb_w  = (HGB_abn_base * w).where(den_hb, 0).sum()
pct_hb_w  = float(num_hb_w / den_hb_w * 100)

fully_norm_base = (m["fully_normal_strict"] == True).astype(int)
den_all5_w = w.where(present_all5, 0).sum()
num_all5_w = (fully_norm_base * w).where(present_all5, 0).sum()
pct_all5_w_base = float(num_all5_w / den_all5_w * 100)

# --- LIBERAL Hb (Hb < 11.5) ---
HGB_abn_115 = (m["hb_gdl"] < 11.5).fillna(False).astype(int)

# recompute "fully normal" under the new Hb rule (other 4 flags unchanged)
none_abn_others = (~m["BMI_abn"]) & (~m["BP_abn"]) & (~m["GLU_abn"]) & (~m["WAIST_abn"])
fully_norm_115 = (present_all5 & none_abn_others & ~(m["hb_gdl"] < 11.5)).astype(int)

num_hb_w_115 = (HGB_abn_115 * w).where(den_hb, 0).sum()
pct_hb_w_115 = float(num_hb_w_115 / den_hb_w * 100)

num_all5_w_115 = (fully_norm_115 * w).where(present_all5, 0).sum()
pct_all5_w_115 = float(num_all5_w_115 / den_all5_w * 100)

print("Hb abnormal (weighted):")
print(f"  Base Hb<12.0  : {pct_hb_w:5.2f}%")
print(f"  Liberal Hb<11.5: {pct_hb_w_115:5.2f}%")

print("\nFully normal across all 5 (weighted, among all five present):")
print(f"  Base Hb<12.0  : {pct_all5_w_base:5.2f}%")
print(f"  Liberal Hb<11.5: {pct_all5_w_115:5.2f}%")


Hb abnormal (weighted):
  Base Hb<12.0  : 57.89%
  Liberal Hb<11.5: 45.35%

Fully normal across all 5 (weighted, among all five present):
  Base Hb<12.0  : 18.25%
  Liberal Hb<11.5: 24.25%


In [18]:
import pandas as pd

m = pd.read_parquet("nfhs5_women_flags.parquet")
w = pd.to_numeric(m["v005"], errors="coerce") / 1_000_000

# Denominators
den_bp = m["sbp_clean"].notna() | m["dbp_clean"].notna()
present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)

# --- BASE BP (140/90) ---
bp_abn_base = m["BP_abn"].fillna(False).astype(int)
den_bp_w = w.where(den_bp, 0).sum()
num_bp_w = (bp_abn_base * w).where(den_bp, 0).sum()
pct_bp_base = float(num_bp_w / den_bp_w * 100)

# --- TIGHT BP (130/80) ---
bp_abn_13080 = ((m["sbp_clean"] >= 130) | (m["dbp_clean"] >= 80)).fillna(False).astype(int)
num_bp_w_13080 = (bp_abn_13080 * w).where(den_bp, 0).sum()
pct_bp_13080 = float(num_bp_w_13080 / den_bp_w * 100)

# Fully normal among those with all five present
# For base: the saved flag (already 140/90, Hb<12)
fully_norm_base = (m["fully_normal_strict"] == True).astype(int)
den_all5_w = w.where(present_all5, 0).sum()
num_all5_w_base = (fully_norm_base * w).where(present_all5, 0).sum()
pct_all5_base = float(num_all5_w_base / den_all5_w * 100)

# For 130/80: recompute only the BP part; other 4 flags unchanged
none_abn_others = (~m["BMI_abn"]) & (~m["GLU_abn"]) & (~m["WAIST_abn"]) & (~m["HGB_abn"])
fully_norm_bp13080 = (present_all5 & none_abn_others & ~( (m["sbp_clean"] >= 130) | (m["dbp_clean"] >= 80) )).astype(int)
num_all5_w_bp13080 = (fully_norm_bp13080 * w).where(present_all5, 0).sum()
pct_all5_bp13080 = float(num_all5_w_bp13080 / den_all5_w * 100)

print("BP abnormal (weighted):")
print(f"  Base BP≥140/90 : {pct_bp_base:5.2f}%")
print(f"  Tight BP≥130/80: {pct_bp_13080:5.2f}%")

print("\nFully normal across all 5 (weighted, among all five present):")
print(f"  Base (with 140/90): {pct_all5_base:5.2f}%")
print(f"  With 130/80      : {pct_all5_bp13080:5.2f}%")


BP abnormal (weighted):
  Base BP≥140/90 :  9.88%
  Tight BP≥130/80: 39.11%

Fully normal across all 5 (weighted, among all five present):
  Base (with 140/90): 18.25%
  With 130/80      : 13.11%


In [19]:
import pandas as pd

m = pd.read_parquet("nfhs5_women_flags.parquet")
w = pd.to_numeric(m["v005"], errors="coerce") / 1_000_000

present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)
den_all5_w = w.where(present_all5, 0).sum()

#  BMI/Waist/Glucose as in base flags only vary Hb and BP cutoffs here
base_other_ok = (~m["BMI_abn"]) & (~m["GLU_abn"]) & (~m["WAIST_abn"])

def calc(bp_sbp_ge, bp_dbp_ge, hb_lt):
    # BP abnormal under this cutoff
    bp_abn = ((m["sbp_clean"] >= bp_sbp_ge) | (m["dbp_clean"] >= bp_dbp_ge)).fillna(False).astype(int)
    den_bp = (m["sbp_clean"].notna() | m["dbp_clean"].notna())
    num_bp_w = (bp_abn * w).where(den_bp, 0).sum()
    den_bp_w = w.where(den_bp, 0).sum()
    pct_bp = float(num_bp_w / den_bp_w * 100) if den_bp_w > 0 else 0.0

    # Hb abnormal under this cutoff
    hb_abn = (m["hb_gdl"] < hb_lt).fillna(False).astype(int)
    den_hb = m["hb_gdl"].notna()
    num_hb_w = (hb_abn * w).where(den_hb, 0).sum()
    den_hb_w = w.where(den_hb, 0).sum()
    pct_hb = float(num_hb_w / den_hb_w * 100) if den_hb_w > 0 else 0.0

    # Fully normal across all 5 with these Hb/BP rules (other three fixed at base)
    fully_norm = (present_all5 & base_other_ok & ~(bp_abn.astype(bool)) & ~(m["hb_gdl"] < hb_lt)).astype(int)
    num_all5_w = (fully_norm * w).where(present_all5, 0).sum()
    pct_all5 = float(num_all5_w / den_all5_w * 100) if den_all5_w > 0 else 0.0

    return round(pct_bp,2), round(pct_hb,2), round(pct_all5,2)

rows = []
scenarios = [
    ("BP≥140/90, Hb<12.0", 140, 90, 12.0),
    ("BP≥140/90, Hb<11.5", 140, 90, 11.5),
    ("BP≥130/80, Hb<12.0", 130, 80, 12.0),
    ("BP≥130/80, Hb<11.5", 130, 80, 11.5),
]
for label, sbp, dbp, hb in scenarios:
    bp, hb_, all5 = calc(sbp, dbp, hb)
    rows.append([label, bp, hb_, all5])

out = pd.DataFrame(rows, columns=["Scenario","BP abnormal % (w)","Hb abnormal % (w)","Fully normal % (w, all 5 present)"])
print(out.to_string(index=False))


          Scenario  BP abnormal % (w)  Hb abnormal % (w)  Fully normal % (w, all 5 present)
BP≥140/90, Hb<12.0               9.88              57.89                              18.25
BP≥140/90, Hb<11.5               9.88              45.35                              24.25
BP≥130/80, Hb<12.0              39.11              57.89                              13.11
BP≥130/80, Hb<11.5              39.11              45.35                              17.64


In [20]:
yaml_text = """threshold_sets:
  - name: base
    bmi_abn_ge: 23.0
    bp_sbp_ge: 140
    bp_dbp_ge: 90
    glucose_abn_ge: 200
    waist_f_ge_cm: 80
    hb_f_lt_gdl: 12.0

  - name: liberal_hgb
    bmi_abn_ge: 23.0
    bp_sbp_ge: 140
    bp_dbp_ge: 90
    glucose_abn_ge: 200
    waist_f_ge_cm: 80
    hb_f_lt_gdl: 11.5

  - name: tight_bp
    bmi_abn_ge: 23.0
    bp_sbp_ge: 130
    bp_dbp_ge: 80
    glucose_abn_ge: 200
    waist_f_ge_cm: 80
    hb_f_lt_gdl: 12.0

  - name: tight_bp__liberal_hgb
    bmi_abn_ge: 23.0
    bp_sbp_ge: 130
    bp_dbp_ge: 80
    glucose_abn_ge: 200
    waist_f_ge_cm: 80
    hb_f_lt_gdl: 11.5
"""
with open("thresholds.yml", "w", encoding="utf-8") as f:
    f.write(yaml_text)
print("Wrote thresholds.yml")


Wrote thresholds.yml


In [21]:
# thresholds.yml and compute weighted stats for a chosen scenario

import pandas as pd
import sys

#  read YAML
try:
    import yaml
except Exception:
    
    %pip install pyyaml
    import yaml


m = pd.read_parquet("nfhs5_women_biomarkers.parquet")
w = pd.to_numeric(m["v005"], errors="coerce") / 1_000_000


den_bmi   = m["bmi"].notna()
den_bp    = m["sbp_clean"].notna() | m["dbp_clean"].notna()
den_glu   = m["glucose_mgdl"].notna()
den_waist = m["waist_cm"].notna()
den_hb    = m["hb_gdl"].notna()
present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)
den_all5_w = w.where(present_all5, 0).sum()

def wshare_bool(flag_bool, denom_mask):
    """Weighted % with proper denominator (who has that metric)."""
    num = (flag_bool.astype(int) * w).where(denom_mask, 0).sum()
    den = w.where(denom_mask, 0).sum()
    return (float(num/den*100) if den>0 else 0.0)

def run_scenario(scn_name="base"):
    
    with open("thresholds.yml", "r", encoding="utf-8") as f:
        cfg_all = yaml.safe_load(f)
    names = [s["name"] for s in cfg_all["threshold_sets"]]
    if scn_name not in names:
        print("Available scenarios:", names)
        raise SystemExit(f"Scenario '{scn_name}' not found.")
    cfg = {k:v for s in cfg_all["threshold_sets"] if s["name"]==scn_name for k,v in s.items() if k!="name"}

    
    BMI_abn   = (m["bmi"]          >= cfg["bmi_abn_ge"]).fillna(False)
    BP_abn    = ((m["sbp_clean"]   >= cfg["bp_sbp_ge"]) | (m["dbp_clean"] >= cfg["bp_dbp_ge"])).fillna(False)
    GLU_abn   = (m["glucose_mgdl"] >= cfg["glucose_abn_ge"]).fillna(False)
    WAIST_abn = (m["waist_cm"]     >= cfg["waist_f_ge_cm"]).fillna(False)
    HGB_abn   = (m["hb_gdl"]       <  cfg["hb_f_lt_gdl"]).fillna(False)

    # 4) Weighted per-metric abnormality
    out_rows = []
    out_rows.append(("BMI abnormal",        round(wshare_bool(BMI_abn,   den_bmi),   2)))
    out_rows.append(("BP abnormal",         round(wshare_bool(BP_abn,    den_bp),    2)))
    out_rows.append(("Glucose abnormal",    round(wshare_bool(GLU_abn,   den_glu),   2)))
    out_rows.append(("Waist abnormal",      round(wshare_bool(WAIST_abn, den_waist), 2)))
    out_rows.append(("Hemoglobin abnormal", round(wshare_bool(HGB_abn,   den_hb),    2)))
    summary = pd.DataFrame(out_rows, columns=["Metric","Abnormal % (weighted)"])

    # 5) Weighted "fully normal" among those with all five present
    none_abn = ~(BMI_abn | BP_abn | GLU_abn | WAIST_abn | HGB_abn)
    fully_norm = (present_all5 & none_abn)
    num_all5_w = (fully_norm.astype(int) * w).where(present_all5, 0).sum()
    pct_all5   = round(float(num_all5_w / den_all5_w * 100), 2) if den_all5_w>0 else 0.0

    print(f"\nScenario: {scn_name}")
    print(summary.to_string(index=False))
    print(f"\nFully normal % (weighted, among all 5 present): {pct_all5:.2f}%")
    return summary, pct_all5


_ = run_scenario("base")



Scenario: base
             Metric  Abnormal % (weighted)
       BMI abnormal                  37.95
        BP abnormal                   9.88
   Glucose abnormal                   1.42
     Waist abnormal                  41.04
Hemoglobin abnormal                  57.89

Fully normal % (weighted, among all 5 present): 18.25%


In [22]:
run_scenario("liberal_hgb")
run_scenario("tight_bp")
run_scenario("tight_bp__liberal_hgb")



Scenario: liberal_hgb
             Metric  Abnormal % (weighted)
       BMI abnormal                  37.95
        BP abnormal                   9.88
   Glucose abnormal                   1.42
     Waist abnormal                  41.04
Hemoglobin abnormal                  45.35

Fully normal % (weighted, among all 5 present): 24.25%

Scenario: tight_bp
             Metric  Abnormal % (weighted)
       BMI abnormal                  37.95
        BP abnormal                  39.11
   Glucose abnormal                   1.42
     Waist abnormal                  41.04
Hemoglobin abnormal                  57.89

Fully normal % (weighted, among all 5 present): 13.11%

Scenario: tight_bp__liberal_hgb
             Metric  Abnormal % (weighted)
       BMI abnormal                  37.95
        BP abnormal                  39.11
   Glucose abnormal                   1.42
     Waist abnormal                  41.04
Hemoglobin abnormal                  45.35

Fully normal % (weighted, among all 5

(                Metric  Abnormal % (weighted)
 0         BMI abnormal                  37.95
 1          BP abnormal                  39.11
 2     Glucose abnormal                   1.42
 3       Waist abnormal                  41.04
 4  Hemoglobin abnormal                  45.35,
 17.64)

In [23]:
import pandas as pd, yaml


m = pd.read_parquet("nfhs5_women_biomarkers.parquet")
w = pd.to_numeric(m["v005"], errors="coerce") / 1_000_000


den_bmi   = m["bmi"].notna()
den_bp    = m["sbp_clean"].notna() | m["dbp_clean"].notna()
den_glu   = m["glucose_mgdl"].notna()
den_waist = m["waist_cm"].notna()
den_hb    = m["hb_gdl"].notna()
present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)
den_all5_w = w.where(present_all5, 0).sum()

def wshare_bool(flag_bool, denom_mask):
    num = (flag_bool.astype(int) * w).where(denom_mask, 0).sum()
    den = w.where(denom_mask, 0).sum()
    return float(num/den*100) if den>0 else 0.0

with open("thresholds.yml","r",encoding="utf-8") as f:
    cfg_all = yaml.safe_load(f)["threshold_sets"]

rows = []
for scn in cfg_all:
    name = scn["name"]
    
    BMI_abn   = (m["bmi"]          >= scn["bmi_abn_ge"]).fillna(False)
    BP_abn    = ((m["sbp_clean"]   >= scn["bp_sbp_ge"]) | (m["dbp_clean"] >= scn["bp_dbp_ge"])).fillna(False)
    GLU_abn   = (m["glucose_mgdl"] >= scn["glucose_abn_ge"]).fillna(False)
    WAIST_abn = (m["waist_cm"]     >= scn["waist_f_ge_cm"]).fillna(False)
    HGB_abn   = (m["hb_gdl"]       <  scn["hb_f_lt_gdl"]).fillna(False)

    # Weighted % per metric
    out = {
        "scenario": name,
        "BMI_abn_w%":        round(wshare_bool(BMI_abn,   den_bmi),   2),
        "BP_abn_w%":         round(wshare_bool(BP_abn,    den_bp),    2),
        "GLU_abn_w%":        round(wshare_bool(GLU_abn,   den_glu),   2),
        "WAIST_abn_w%":      round(wshare_bool(WAIST_abn, den_waist), 2),
        "HGB_abn_w%":        round(wshare_bool(HGB_abn,   den_hb),    2),
    }

    # Fully normal (weighted, among all five present)
    none_abn = ~(BMI_abn | BP_abn | GLU_abn | WAIST_abn | HGB_abn)
    fully_norm = (present_all5 & none_abn)
    num_all5_w = (fully_norm.astype(int) * w).where(present_all5, 0).sum()
    out["Fully_normal_w%_all5"] = round(float(num_all5_w / den_all5_w * 100), 2) if den_all5_w>0 else 0.0

    rows.append(out)

sens = pd.DataFrame(rows)
sens.to_csv("section1_threshold_sensitivity.csv", index=False)
print("Saved: section1_threshold_sensitivity.csv")
print(sens)


Saved: section1_threshold_sensitivity.csv
                scenario  BMI_abn_w%  BP_abn_w%  GLU_abn_w%  WAIST_abn_w%  \
0                   base       37.95       9.88        1.42         41.04   
1            liberal_hgb       37.95       9.88        1.42         41.04   
2               tight_bp       37.95      39.11        1.42         41.04   
3  tight_bp__liberal_hgb       37.95      39.11        1.42         41.04   

   HGB_abn_w%  Fully_normal_w%_all5  
0       57.89                 18.25  
1       45.35                 24.25  
2       57.89                 13.11  
3       45.35                 17.64  


In [24]:
import pandas as pd, yaml


m = pd.read_parquet("nfhs5_women_biomarkers.parquet")
w = pd.to_numeric(m["v005"], errors="coerce") / 1_000_000

# Use only women who have all five measures
present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)
m = m.loc[present_all5].copy()
w = w.loc[present_all5]

# Read thresholds and pick the base scenario
with open("thresholds.yml","r",encoding="utf-8") as f:
    cfg_all = yaml.safe_load(f)["threshold_sets"]
cfg = next(s for s in cfg_all if s["name"]=="base")

# Build flags under the base scenario
BMI = (m["bmi"]          >= cfg["bmi_abn_ge"])
BP  = (m["sbp_clean"]    >= cfg["bp_sbp_ge"]) | (m["dbp_clean"] >= cfg["bp_dbp_ge"])
GLU = (m["glucose_mgdl"] >= cfg["glucose_abn_ge"])
WAI = (m["waist_cm"]     >= cfg["waist_f_ge_cm"])
HGB = (m["hb_gdl"]       <  cfg["hb_f_lt_gdl"])

flags = pd.DataFrame({"BMI":BMI, "BP":BP, "GLU":GLU, "WAIST":WAI, "HGB":HGB})

# Encode each row combination as a label like "BMI+WAIST+HGB" (or "NONE" if all False)
def combo_label(row):
    names = [k for k,v in row.items() if v]
    return "+".join(names) if names else "NONE"

combos = flags.apply(combo_label, axis=1)

# Weighted counts & shares among all-five-present
den_w = w.sum()
out = (pd.DataFrame({"combo":combos, "w":w})
         .groupby("combo", as_index=False)["w"].sum()
         .assign(pct=lambda d: d["w"]/den_w*100,
                 k_abn=lambda d: d["combo"].str.count(r"\+") + (d["combo"]!="NONE").astype(int))
         .sort_values(["k_abn","pct"], ascending=[False, False]))

# Show the top 15 combinations
top15 = out.head(15).copy()
top15["pct"] = top15["pct"].round(2)
top15.rename(columns={"w":"weighted_count","pct":"weighted_pct","k_abn":"#abnormal"}, inplace=True)

print("Top 15 co-abnormality combinations (weighted, base scenario; among women with all five measures):")
print(top15.to_string(index=False))


Top 15 co-abnormality combinations (weighted, base scenario; among women with all five measures):
               combo  weighted_count  weighted_pct  #abnormal
BMI+BP+GLU+WAIST+HGB     1107.276381          0.16          5
    BMI+BP+WAIST+HGB    14283.990785          2.12          4
   BMI+GLU+WAIST+HGB     2182.510784          0.32          4
    BMI+BP+GLU+WAIST     1347.570634          0.20          4
    BP+GLU+WAIST+HGB      170.360220          0.03          4
      BMI+BP+GLU+HGB       90.104469          0.01          4
       BMI+WAIST+HGB    86640.024993         12.84          3
        BMI+BP+WAIST    15940.022946          2.36          3
        BP+WAIST+HGB     3348.574009          0.50          3
          BMI+BP+HGB     2856.860210          0.42          3
       BMI+GLU+WAIST     2108.110935          0.31          3
       GLU+WAIST+HGB      320.907125          0.05          3
         BMI+GLU+HGB      176.535526          0.03          3
          BP+GLU+HGB      148.6396

In [25]:
import pandas as pd, yaml
from math import prod

# Load data + weights
m = pd.read_parquet("nfhs5_women_biomarkers.parquet")
w = pd.to_numeric(m["v005"], errors="coerce") / 1_000_000

# Keep only women with all 5 measures (so exact combos are well-defined)
present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)
m = m.loc[present_all5].copy()
w = w.loc[present_all5]

# Read thresholds  use the "base" scenario for now
with open("thresholds.yml","r",encoding="utf-8") as f:
    cfg = next(s for s in yaml.safe_load(f)["threshold_sets"] if s["name"]=="base")

# Build abnormal flags for base cut-offs
flags = pd.DataFrame({
    "BMI":  (m["bmi"]          >= cfg["bmi_abn_ge"]),
    "BP":   (m["sbp_clean"]    >= cfg["bp_sbp_ge"]) | (m["dbp_clean"] >= cfg["bp_dbp_ge"]),
    "GLU":  (m["glucose_mgdl"] >= cfg["glucose_abn_ge"]),
    "WAIST":(m["waist_cm"]     >= cfg["waist_f_ge_cm"]),
    "HGB":  (m["hb_gdl"]       <  cfg["hb_f_lt_gdl"]),
})

# Weighted marginal probabilities p(metric abnormal)
den_w = w.sum()
p = {}
for col in flags.columns:
    p[col] = ((flags[col].astype(int) * w).sum()) / den_w

# Encode each row’s exact combo label ("BMI+WAIST+HGB" or "NONE")
def combo_label(row):
    ks = [k for k,v in row.items() if v]
    return "+".join(ks) if ks else "NONE"

combos = flags.apply(combo_label, axis=1)

# Observed weighted % by combo
obs = (pd.DataFrame({"combo": combos, "w": w})
         .groupby("combo", as_index=False)["w"].sum())
obs["obs_pct"] = obs["w"] / den_w * 100

# Expected % under independence for the exact combo (product of p or (1-p))
metrics = list(flags.columns)
def expected_pct(combo_name):
    if combo_name=="NONE":
        # all five normal
        terms = [(1-p[k]) for k in metrics]
    else:
        abn = set(combo_name.split("+"))
        terms = [ (p[k] if k in abn else (1-p[k])) for k in metrics ]
    return prod(terms) * 100

obs["exp_pct"] = obs["combo"].apply(expected_pct)

# Enrichment ratio 
obs["enrichment"] = obs["obs_pct"] / obs["exp_pct"].where(obs["exp_pct"]>0, float("nan"))
obs["#abnormal"]  = obs["combo"].apply(lambda s: 0 if s=="NONE" else s.count("+")+1)

# Focus table: drop ultra-rare noise (expected < 0.05% or observed < 0.05%)
focus = obs[(obs["exp_pct"]>=0.05) & (obs["obs_pct"]>=0.05)].copy()
focus = focus.sort_values(["enrichment","obs_pct"], ascending=[False, False])

# Show top 15 by enrichment
out = focus[["combo","#abnormal","obs_pct","exp_pct","enrichment"]].head(15).copy()
out["obs_pct"] = out["obs_pct"].round(2)
out["exp_pct"] = out["exp_pct"].round(2)
out["enrichment"] = out["enrichment"].round(2)

print("More-than-chance co-abnormalities (weighted, base; among women with all five measures):")
print(out.to_string(index=False))


More-than-chance co-abnormalities (weighted, base; among women with all five measures):
            combo  #abnormal  obs_pct  exp_pct  enrichment
     BMI+BP+WAIST          3     2.36     0.61        3.90
    BMI+GLU+WAIST          3     0.31     0.08        3.74
BMI+GLU+WAIST+HGB          4     0.32     0.11        2.82
 BMI+BP+WAIST+HGB          4     2.12     0.83        2.55
        BMI+WAIST          2    10.66     5.82        1.83
    BMI+WAIST+HGB          3    12.84     7.99        1.61
              HGB          1    29.22    19.00        1.54
             NONE          0    18.25    13.83        1.32
               BP          1     1.12     1.44        0.78
           BP+HGB          2     1.49     1.98        0.75
           BMI+BP          2     0.44     0.87        0.51
         BP+WAIST          2     0.49     1.00        0.49
        WAIST+HGB          2     6.27    13.15        0.48
            WAIST          1     4.53     9.57        0.47
              BMI          

In [26]:
import pandas as pd, yaml, numpy as np


m = pd.read_parquet("nfhs5_women_biomarkers.parquet")
w = pd.to_numeric(m["v005"], errors="coerce") / 1_000_000
present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)

m = m.loc[present_all5].copy()
w = w.loc[present_all5]
den_w = w.sum()


strata = m["v024"].astype("Int16").astype(str) + "_" + m["v025"].astype("Int8").astype(str)
strata = strata.to_numpy()

# Base thresholds
with open("thresholds.yml","r",encoding="utf-8") as f:
    base = next(s for s in yaml.safe_load(f)["threshold_sets"] if s["name"]=="base")

BMI = (m["bmi"]          >= base["bmi_abn_ge"]).to_numpy()
BP  = ((m["sbp_clean"]   >= base["bp_sbp_ge"]) | (m["dbp_clean"] >= base["bp_dbp_ge"])).to_numpy()
GLU = (m["glucose_mgdl"] >= base["glucose_abn_ge"]).to_numpy()
WAI = (m["waist_cm"]     >= base["waist_f_ge_cm"]).to_numpy()
HGB = (m["hb_gdl"]       <  base["hb_f_lt_gdl"]).to_numpy()
weights = w.to_numpy()

# Observed combo % helper
def pct_combo(mask):
    return float(weights[mask].sum() / den_w * 100)

# Observed table 
import itertools
names = ["BMI","BP","GLU","WAIST","HGB"]
flags = {"BMI":BMI, "BP":BP, "GLU":GLU, "WAIST":WAI, "HGB":HGB}

def combo_mask(combo_names, f=flags):
    msk = np.ones(len(weights), dtype=bool)
    for k in names:  # exactly this combo: abns in combo, normals otherwise
        if k in combo_names:
            msk &= f[k]
        else:
            msk &= ~f[k]
    return msk

# Compute observed %s for all combos
obs = []
for r in range(0,6):
    for comb in itertools.combinations(names, r):
        label = "+".join(comb) if comb else "NONE"
        msk = combo_mask(set(comb))
        obs_pct = pct_combo(msk)
        obs.append((label, r, obs_pct))
obs_df = pd.DataFrame(obs, columns=["combo","#abnormal","obs_pct"]).sort_values("obs_pct", ascending=False)

# Keep enriched combos (from your earlier independence table) and with obs_pct >= 0.2% and not NONE
enriched_candidates = set(["BMI+BP+WAIST","BMI+GLU+WAIST","BMI+GLU+WAIST+HGB","BMI+BP+WAIST+HGB","BMI+WAIST","BMI+WAIST+HGB"])
test_combos = [c for c in obs_df["combo"].tolist() if (c in enriched_candidates) and (c!="NONE") and (obs_df.loc[obs_df["combo"]==c,"obs_pct"].values[0] >= 0.2)]

# --- Permutation within strata
rng = np.random.default_rng(123)
def shuffle_within_groups(arr, groups):
    arr = arr.copy()
    for g in np.unique(groups):
        idx = np.where(groups==g)[0]
        rng.shuffle(arr[idx])
    return arr

n_perm = 80  # adjust for speed vs precision

perm_max = {c:[] for c in test_combos}
for _ in range(n_perm):
    BMI_p = shuffle_within_groups(BMI, strata)
    BP_p  = shuffle_within_groups(BP, strata)
    GLU_p = shuffle_within_groups(GLU, strata)
    WAI_p = shuffle_within_groups(WAI, strata)
    HGB_p = shuffle_within_groups(HGB, strata)
    f = {"BMI":BMI_p, "BP":BP_p, "GLU":GLU_p, "WAIST":WAI_p, "HGB":HGB_p}
    for c in test_combos:
        comb = set(c.split("+")) if c!="NONE" else set()
        msk = combo_mask(comb, f=f)
        perm_max[c].append(pct_combo(msk))

# One-sided p-values (Pr[perm ≥ obs]); add +1 smoothing
results = []
for c in test_combos:
    obs_c = float(obs_df.loc[obs_df["combo"]==c,"obs_pct"].values[0])
    per = np.array(perm_max[c], dtype=float)
    pval = ( (per >= obs_c).sum() + 1 ) / (len(per) + 1)
    results.append((c, obs_c, float(per.mean()), float(per.std(ddof=1)), float(pval)))

res = pd.DataFrame(results, columns=["combo","obs_pct","perm_mean_pct","perm_sd_pct","p_perm"]).sort_values("p_perm")
# Benjamini–Hochberg FDR
p = res["p_perm"].to_numpy()
order = np.argsort(p)
rank = np.arange(1, len(p)+1)
q = p[order] * len(p) / rank
q = np.minimum.accumulate(q[::-1])[::-1]
q_adj = np.empty_like(q); q_adj[order] = np.clip(q, 0, 1)
res["q_fdr"] = q_adj

print("Permutation test (state×urban shuffled), base scenario")
print(res.to_string(index=False))


Permutation test (state×urban shuffled), base scenario
            combo   obs_pct  perm_mean_pct  perm_sd_pct  p_perm  q_fdr
    BMI+WAIST+HGB 12.838606      12.838606 3.575128e-15     1.0    1.0
        BMI+WAIST 10.663626      10.663626 1.787564e-15     1.0    1.0
     BMI+BP+WAIST  2.362045       2.362045 4.468911e-16     1.0    1.0
 BMI+BP+WAIST+HGB  2.116649       2.116649 4.468911e-16     1.0    1.0
BMI+GLU+WAIST+HGB  0.323412       0.323412 5.586138e-17     1.0    1.0
    BMI+GLU+WAIST  0.312387       0.312387 0.000000e+00     1.0    1.0


In [27]:
import pandas as pd, yaml
from math import prod


m = pd.read_parquet("nfhs5_women_biomarkers.parquet")
w = pd.to_numeric(m["v005"], errors="coerce") / 1_000_000
present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)
m = m.loc[present_all5].copy(); w = w.loc[present_all5]; den_w = w.sum()

# Base thresholds
with open("thresholds.yml","r",encoding="utf-8") as f:
    base = next(s for s in yaml.safe_load(f)["threshold_sets"] if s["name"]=="base")

flags = pd.DataFrame({
    "BMI":  (m["bmi"]          >= base["bmi_abn_ge"]),
    "BP":   (m["sbp_clean"]    >= base["bp_sbp_ge"]) | (m["dbp_clean"] >= base["bp_dbp_ge"]),
    "GLU":  (m["glucose_mgdl"] >= base["glucose_abn_ge"]),
    "WAIST":(m["waist_cm"]     >= base["waist_f_ge_cm"]),
    "HGB":  (m["hb_gdl"]       <  base["hb_f_lt_gdl"]),
})

# Weighted marginals
p = {col: (flags[col].astype(int) * w).sum() / den_w for col in flags.columns}

# Label exact combo per row
def combo_label(row):
    ks = [k for k,v in row.items() if v]
    return "+".join(ks) if ks else "NONE"
combos = flags.apply(combo_label, axis=1)

# Observed weighted % by combo
obs = (pd.DataFrame({"combo": combos, "w": w})
         .groupby("combo", as_index=False)["w"].sum())
obs["obs_pct"] = obs["w"] / den_w * 100


metrics = list(flags.columns)
def expected_pct(name):
    if name=="NONE":
        return prod([(1-p[k]) for k in metrics]) * 100
    abn = set(name.split("+"))
    return prod([(p[k] if k in abn else (1-p[k])) for k in metrics]) * 100

obs["exp_pct"] = obs["combo"].apply(expected_pct)
obs["enrichment"] = obs["obs_pct"] / obs["exp_pct"].where(obs["exp_pct"]>0)
obs["#abnormal"]  = obs["combo"].apply(lambda s: 0 if s=="NONE" else s.count("+")+1)


table = (obs.sort_values(["#abnormal","obs_pct"], ascending=[False, False])
           .assign(obs_pct=lambda d: d["obs_pct"].round(2),
                   exp_pct=lambda d: d["exp_pct"].round(2),
                   enrichment=lambda d: d["enrichment"].round(2)))
table.to_csv("section1_combos_obs_vs_expected_base.csv", index=False)
print("Saved: section1_combos_obs_vs_expected_base.csv")
print(table.head(15).to_string(index=False))


Saved: section1_combos_obs_vs_expected_base.csv
               combo            w  obs_pct  exp_pct  enrichment  #abnormal
BMI+BP+GLU+WAIST+HGB  1107.276381     0.16     0.01       13.74          5
    BMI+BP+WAIST+HGB 14283.990785     2.12     0.83        2.55          4
   BMI+GLU+WAIST+HGB  2182.510784     0.32     0.11        2.82          4
    BMI+BP+GLU+WAIST  1347.570634     0.20     0.01       22.97          4
    BP+GLU+WAIST+HGB   170.360220     0.03     0.02        1.28          4
      BMI+BP+GLU+HGB    90.104469     0.01     0.02        0.77          4
       BMI+WAIST+HGB 86640.024993    12.84     7.99        1.61          3
        BMI+BP+WAIST 15940.022946     2.36     0.61        3.90          3
        BP+WAIST+HGB  3348.574009     0.50     1.37        0.36          3
          BMI+BP+HGB  2856.860210     0.42     1.20        0.35          3
       BMI+GLU+WAIST  2108.110935     0.31     0.08        3.74          3
       GLU+WAIST+HGB   320.907125     0.05     0.19 

In [33]:
import os, pandas as pd, yaml
import matplotlib.pyplot as plt


m = pd.read_parquet("nfhs5_women_biomarkers.parquet")
w = pd.to_numeric(m["v005"], errors="coerce") / 1_000_000
present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)
m = m.loc[present_all5].copy()
w = w.loc[present_all5]

# 2) Base thresholds  abnormal flags
with open("thresholds.yml","r",encoding="utf-8") as f:
    base = next(s for s in yaml.safe_load(f)["threshold_sets"] if s["name"]=="base")

flags = pd.DataFrame({
    "BMI":   (m["bmi"]          >= base["bmi_abn_ge"]),
    "BP":    (m["sbp_clean"]    >= base["bp_sbp_ge"]) | (m["dbp_clean"] >= base["bp_dbp_ge"]),
    "GLU":   (m["glucose_mgdl"] >= base["glucose_abn_ge"]),
    "WAIST": (m["waist_cm"]     >= base["waist_f_ge_cm"]),
    "HGB":   (m["hb_gdl"]       <  base["hb_f_lt_gdl"]),
})

# 3) Build human-readable combo labels per row 
combo_labels = flags.apply(lambda r: "+".join([c for c in ["BMI","BP","GLU","WAIST","HGB"] if r[c]]), axis=1)
mask_any = flags.any(axis=1)
combo_labels = combo_labels[mask_any]
w_any = w[mask_any]

# 4) Aggregate weighted counts per combo and save CSV
intersections = (
    pd.DataFrame({"combo": combo_labels, "w": w_any})
      .groupby("combo", as_index=False)["w"].sum()
      .assign(weighted_pct=lambda d: d["w"] / w.sum() * 100)
      .sort_values("w", ascending=False)
)

intersections.to_csv("section1_upset_intersections_base.csv", index=False)
print("Saved table → section1_upset_intersections_base.csv")
print(intersections.head(10))

# 5) Fallback figure: Top-15 combos bar chart (weighted %)
os.makedirs("figs", exist_ok=True)
top15 = intersections.head(15).iloc[::-1]  # reverse for horizontal bars (small→large)
plt.figure(figsize=(8,6))
plt.barh(top15["combo"], top15["weighted_pct"])
plt.xlabel("Weighted % (among women with all 5 measures)")
plt.title("NFHS-5 (Women): Top-15 abnormality combinations — Base thresholds")
plt.tight_layout()
out_path = "figs/top15_combos_base.png"
plt.savefig(out_path, dpi=200)
plt.close()
print("Saved fallback figure →", out_path)


Saved table → section1_upset_intersections_base.csv
               combo              w  weighted_pct
28               HGB  197196.246856     29.221193
15     BMI+WAIST+HGB   86640.024993     12.838606
14         BMI+WAIST   71962.397106     10.663626
30         WAIST+HGB   42282.459273      6.265555
29             WAIST   30598.991736      4.534260
13           BMI+HGB   29170.547007      4.322588
0                BMI   23887.419406      3.539717
7       BMI+BP+WAIST   15940.022946      2.362045
8   BMI+BP+WAIST+HGB   14283.990785      2.116649
21            BP+HGB   10034.239143      1.486907
Saved fallback figure → figs/top15_combos_base.png


In [34]:
import pandas as pd, yaml

# --- write an extended thresholds.yml ---
yaml_text = """threshold_sets:
  - name: base
    bmi_abn_ge: 23.0
    bp_sbp_ge: 140
    bp_dbp_ge: 90
    glucose_abn_ge: 200
    waist_f_ge_cm: 80
    hb_f_lt_gdl: 12.0

  - name: liberal_hgb
    bmi_abn_ge: 23.0
    bp_sbp_ge: 140
    bp_dbp_ge: 90
    glucose_abn_ge: 200
    waist_f_ge_cm: 80
    hb_f_lt_gdl: 11.5

  - name: tight_bp
    bmi_abn_ge: 23.0
    bp_sbp_ge: 130
    bp_dbp_ge: 80
    glucose_abn_ge: 200
    waist_f_ge_cm: 80
    hb_f_lt_gdl: 12.0

  - name: tight_bp__liberal_hgb
    bmi_abn_ge: 23.0
    bp_sbp_ge: 130
    bp_dbp_ge: 80
    glucose_abn_ge: 200
    waist_f_ge_cm: 80
    hb_f_lt_gdl: 11.5

  # --- NEW sensitivity presets ---
  - name: bmi25
    bmi_abn_ge: 25.0
    bp_sbp_ge: 140
    bp_dbp_ge: 90
    glucose_abn_ge: 200
    waist_f_ge_cm: 80
    hb_f_lt_gdl: 12.0

  - name: waist88
    bmi_abn_ge: 23.0
    bp_sbp_ge: 140
    bp_dbp_ge: 90
    glucose_abn_ge: 200
    waist_f_ge_cm: 88
    hb_f_lt_gdl: 12.0

  - name: bmi25__waist88
    bmi_abn_ge: 25.0
    bp_sbp_ge: 140
    bp_dbp_ge: 90
    glucose_abn_ge: 200
    waist_f_ge_cm: 88
    hb_f_lt_gdl: 12.0

  - name: bmi25__waist88__tight_bp
    bmi_abn_ge: 25.0
    bp_sbp_ge: 130
    bp_dbp_ge: 80
    glucose_abn_ge: 200
    waist_f_ge_cm: 88
    hb_f_lt_gdl: 12.0

  - name: bmi25__waist88__tight_bp__liberal_hgb
    bmi_abn_ge: 25.0
    bp_sbp_ge: 130
    bp_dbp_ge: 80
    glucose_abn_ge: 200
    waist_f_ge_cm: 88
    hb_f_lt_gdl: 11.5
"""
with open("thresholds.yml","w",encoding="utf-8") as f:
    f.write(yaml_text)

# I tried to recompute the weighted sensitivity table for ALL scenarios above 
m = pd.read_parquet("nfhs5_women_biomarkers.parquet")
w = pd.to_numeric(m["v005"], errors="coerce") / 1_000_000

den_bmi   = m["bmi"].notna()
den_bp    = m["sbp_clean"].notna() | m["dbp_clean"].notna()
den_glu   = m["glucose_mgdl"].notna()
den_waist = m["waist_cm"].notna()
den_hb    = m["hb_gdl"].notna()
present_all5 = m[["bmi","sbp_clean","dbp_clean","glucose_mgdl","waist_cm","hb_gdl"]].notna().all(axis=1)
den_all5_w = w.where(present_all5, 0).sum()

def wshare(flag_bool, denom_mask):
    num = (flag_bool.astype(int) * w).where(denom_mask, 0).sum()
    den = w.where(denom_mask, 0).sum()
    return float(num/den*100) if den>0 else 0.0

cfg_all = yaml.safe_load(open("thresholds.yml","r",encoding="utf-8"))["threshold_sets"]

rows = []
for scn in cfg_all:
    name = scn["name"]
    BMI_abn   = (m["bmi"]          >= scn["bmi_abn_ge"]).fillna(False)
    BP_abn    = ((m["sbp_clean"]   >= scn["bp_sbp_ge"]) | (m["dbp_clean"] >= scn["bp_dbp_ge"])).fillna(False)
    GLU_abn   = (m["glucose_mgdl"] >= scn["glucose_abn_ge"]).fillna(False)
    WAIST_abn = (m["waist_cm"]     >= scn["waist_f_ge_cm"]).fillna(False)
    HGB_abn   = (m["hb_gdl"]       <  scn["hb_f_lt_gdl"]).fillna(False)

    out = {
        "scenario": name,
        "BMI_abn_w%":        round(wshare(BMI_abn,   den_bmi),   2),
        "BP_abn_w%":         round(wshare(BP_abn,    den_bp),    2),
        "GLU_abn_w%":        round(wshare(GLU_abn,   den_glu),   2),
        "WAIST_abn_w%":      round(wshare(WAIST_abn, den_waist), 2),
        "HGB_abn_w%":        round(wshare(HGB_abn,   den_hb),    2),
    }
    none_abn = ~(BMI_abn | BP_abn | GLU_abn | WAIST_abn | HGB_abn)
    fully_norm = (present_all5 & none_abn)
    num_all5_w = (fully_norm.astype(int) * w).where(present_all5, 0).sum()
    out["Fully_normal_w%_all5"] = round(float(num_all5_w / den_all5_w * 100), 2) if den_all5_w>0 else 0.0
    rows.append(out)

sens = pd.DataFrame(rows).sort_values("scenario")
sens.to_csv("section1_threshold_sensitivity.csv", index=False)
print("Saved: section1_threshold_sensitivity.csv")
print(sens)


Saved: section1_threshold_sensitivity.csv
                                scenario  BMI_abn_w%  BP_abn_w%  GLU_abn_w%  \
0                                   base       37.95       9.88        1.42   
4                                  bmi25       23.88       9.88        1.42   
6                         bmi25__waist88       23.88       9.88        1.42   
7               bmi25__waist88__tight_bp       23.88      39.11        1.42   
8  bmi25__waist88__tight_bp__liberal_hgb       23.88      39.11        1.42   
1                            liberal_hgb       37.95       9.88        1.42   
2                               tight_bp       37.95      39.11        1.42   
3                  tight_bp__liberal_hgb       37.95      39.11        1.42   
5                                waist88       37.95       9.88        1.42   

   WAIST_abn_w%  HGB_abn_w%  Fully_normal_w%_all5  
0         41.04       57.89                 18.25  
4         41.04       57.89                 20.55  
6         2